In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from math import isnan
import pickle 

In [16]:
df = pd.read_csv('../datasets/movies-dataset/ratings_small.csv')
metadata = pd.read_csv('../datasets/movies-dataset/movies_metadata.csv')
metadata.id = pd.to_numeric(metadata.id, errors='coerce')

# df.loc[len(df.index) + 1] = [0, 1726, 5, 123]
# df.loc[len(df.index) + 1] = [0, 218, 5, 123]
# df.loc[len(df.index) + 1] = [0, 280, 5, 123]
# df.loc[len(df.index) + 1] = [0, 608, 5, 123]
# df.loc[len(df.index) + 1] = [0, 562, 5, 123]
df.loc[len(df.index) + 1] = [0, 1726, 5, 123]

df_matrix = df.pivot(index="userId", columns="movieId", values="rating").fillna(0)
matrix = csr_matrix(df_matrix)

/var/folders/_q/vmvd_g61081_dvs5kftpmz140000gn/T/ipykernel_66808/1646605096.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv('../datasets/movies-dataset/movies_metadata.csv')


In [17]:
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=21)
knn.fit(matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=21)

In [18]:
distances, user_ids = knn.kneighbors(matrix[0], n_neighbors=21)

In [19]:
similar_users = user_ids[0][1:]
distances = distances[0][1:]

In [21]:
similar_users

array([533, 315, 213,  94,  43,  30, 564, 455, 453, 458, 457, 460, 446,
       448, 447, 456, 450, 459, 449, 452])

In [12]:
movies_to_predict = df_matrix.loc[0][df_matrix.loc[0] == 0].index

In [13]:
matrix_similar_users = df_matrix.loc[similar_users]

In [14]:
predictions = dict()

for movie_id in movies_to_predict:
    movie = matrix_similar_users[movie_id]

    mean = 0
    for i in range(len(similar_users)):
        mean += distances[i] * movie[similar_users[i]]
    
    if mean != 0 and not isnan(mean):
        movie_q = metadata[metadata.id == movie_id]
        if movie_q.shape[0] > 0:
            predictions[movie_id] = mean

In [15]:
dict(sorted(predictions.items(), key=lambda item: item[1], reverse=True))

{587: 49.260653492775,
 480: 46.1092771040121,
 539: 42.78263264295868,
 590: 41.10119528616479,
 110: 38.5864818527961,
 457: 38.501716570552304,
 527: 37.93470396080893,
 500: 36.266726709653305,
 364: 36.084949322084,
 296: 34.53988457637404,
 165: 33.74898537065649,
 593: 32.59281762719731,
 377: 30.982500248487355,
 380: 30.65917938759497,
 595: 29.963769922531622,
 318: 29.259716514385893,
 588: 29.238735203405863,
 339: 29.095585847438223,
 454: 28.953720970265447,
 316: 28.30966300001359,
 150: 28.28346328411734,
 586: 26.8852079374408,
 434: 25.63691596846032,
 292: 24.039755840114623,
 592: 23.1147277172476,
 231: 23.099344907131076,
 344: 22.50797702151206,
 509: 20.39672521038453,
 440: 19.72723249905102,
 185: 19.610507163891384,
 253: 18.87838042886843,
 317: 18.764623777506444,
 329: 17.18192644532255,
 350: 16.920843266067983,
 236: 16.71013181852762,
 266: 15.408033805817727,
 225: 15.383910607879987,
 1721: 15.27322146079765,
 1923: 14.799134223396639,
 349: 14.739313

In [690]:
predictions[1573]

2.857102347531138